In [1]:
import os
import re
import pandas as pd

POSITIVE_DIR = 'data/positive/'
NEGATIVE_DIR = 'data/negative/'

In [2]:
def load_json_files(dir_name):
    files = [f for f in os.listdir(dir_name) if f.endswith('.json')]
    df = pd.DataFrame()
    import json
    for f in files:
        with open(dir_name + f) as json_file:
            data = json.load(json_file)
            for value in data:
                df_aux = {}
                for scores in value:
                    part = scores['part']
                    df_aux['score_' + part] = scores['score']
                    df_aux['position_x_' + part] = scores['position']['x']
                    df_aux['position_y_' + part] = scores['position']['y']
                df = df.append(df_aux, ignore_index=True)
    return df

In [3]:
df_positive = load_json_files(POSITIVE_DIR)
df_negative = load_json_files(NEGATIVE_DIR)
df_positive['class'] = 1
df_negative['class'] = 0
df = df_positive.append(df_negative)

In [4]:
df.head()

,position_x_leftAnkle,position_x_leftEar,position_x_leftElbow,position_x_leftEye,position_x_leftHip,position_x_leftKnee,position_x_leftShoulder,position_x_leftWrist,position_x_nose,position_x_rightAnkle,...,score_nose,score_rightAnkle,score_rightEar,score_rightElbow,score_rightEye,score_rightHip,score_rightKnee,score_rightShoulder,score_rightWrist,class
0,86.689562,475.071675,682.146813,451.837372,421.187606,199.474623,542.782890,81.328669,423.265462,84.529987,...,0.999254,0.034086,0.844763,0.578124,0.999635,0.069375,0.038177,0.979412,0.456421,1
1,594.481567,440.496470,629.739985,424.751727,405.975176,131.047420,511.105264,545.525054,403.456252,500.439976,...,0.998187,0.012726,0.773543,0.448644,0.998898,0.085103,0.080082,0.531657,0.096080,1
2,524.047567,340.148807,538.992868,320.029910,334.361368,313.346987,319.813722,738.636441,287.671730,528.359071,...,0.999554,0.033364,0.932265,0.466368,0.999205,0.078910,0.040251,0.692615,0.773867,1
3,601.338071,355.671068,615.440897,327.842196,312.797671,638.903524,447.185021,411.688589,308.695681,136.831677,...,0.999821,0.020808,0.903296,0.294771,0.999145,0.065031,0.027676,0.773847,0.460311,1
4,101.609927,595.272317,362.193732,570.653489,810.200043,90.434758,662.900077,345.576780,538.733718,104.335907,...,0.999701,0.050357,0.938881,0.606705,0.999309,0.196663,0.283722,0.602111,0.652565,1


In [5]:
X = df.drop('class', axis=1)
y = df['class']

In [6]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20, random_state=20)

In [7]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(max_depth=2, random_state = 123)
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)

score = rf_model.score(X_test, y_test)
print('Score:', score)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != predictions).sum()))

Score: 0.8
Number of mislabeled points out of a total 25 points : 5
